# Setup

In [1]:
%load_ext cython

In [ ]:
# %%cython
import pickle

from sklearn import set_config

set_config(display="diagram")
import numpy as np
import pandas as pd

# import seaborn as sns
# from dataprep.eda import create_report
# from pandas_summary import DataFrameSummary

pd.options.display.max_columns = 50
# pd.describe_option()
pd.options.compute.use_numba = True
pd.options.compute.use_numexpr = True

## Compressing the Data

In [ ]:
%%cython
import pandas as pd

data = pd.read_csv("data/data.csv", index_col=0)

In [ ]:
int64_cols = data.select_dtypes("int64")
for c in int64_cols:
    data[c] = data[c].astype("uint8")

In [ ]:
# fl16_cols = data.select_dtypes(include="float64")

# for c in fl16_cols:
#     data[c] = data[c].astype(np.float32)

In [ ]:
with open("data.pkl", "wb+") as fp:
    pickle.dump(data, fp)

In [ ]:
with open("data.pkl", "rb") as fp:
    dpkl = pickle.load(fp)

In [ ]:
int64_cols = dpkl.select_dtypes("int64")
for c in int64_cols:
    dpkl[c] = dpkl[c].astype("uint8")

In [ ]:
int64_cols = dpkl.select_dtypes("uint8")
for c in int64_cols:
    dpkl[c] = dpkl[c].astype("category")

In [ ]:
na_mat = dpkl.isna()

miss_row = na_mat.sum(axis=1)

dpkl["missing_cols"] = miss_row

In [ ]:
def feature_one_hot(f, df=dpkl):
    f_one_hot = pd.get_dummies(df[f], prefix=str(f))
    df = pd.concat([df, f_one_hot], axis=1)
    df.drop([f], axis=1, inplace=True)
    return df.copy()


cats = [x for x in dpkl.columns if "F_2" in x]
# for c in cats:
#     dpkl = feature_one_hot(c)
spar = pd.DataFrame()
cat_pd = dpkl.loc[:, cats].copy()

for fi in cats:
    new_n = fi.replace("_", "") + "_"
    f_one_hot = pd.get_dummies(dpkl[fi], prefix=new_n)
    spar_ = f_one_hot.astype(pd.SparseDtype(np.uint8, fill_value=0))
    for c in spar_.columns:
        spar[c] = spar_[c]
dpkl = dpkl.drop(cats, axis=1)
for c in spar.columns:
    dpkl[c] = spar[c]

In [ ]:
# "F_2_0".replace("_", "") + "_"
# spar.info()


In [ ]:
int64_cols = dpkl.select_dtypes("int64")
for c in int64_cols:
    dpkl[c] = dpkl[c].astype("uint8")

In [ ]:
fl16_cols = dpkl.select_dtypes(include="float64")

for c in fl16_cols:
    dpkl[c] = dpkl[c].astype(np.float32)

#### Save Compressed Data

In [ ]:
with open("data.pkl", "wb+") as fp:
    pickle.dump(dpkl, fp)

## Load Compressed Data

In [7]:
dpkl

,F_1_0,F_1_1,F_1_2,F_1_3,F_1_4,F_1_5,F_1_6,F_1_7,F_1_8,F_1_9,F_1_10,F_1_11,F_1_12,F_1_13,F_1_14,F_3_0,F_3_1,F_3_2,F_3_3,F_3_4,F_3_5,F_3_6,F_3_7,F_3_8,F_3_9,...,F223__5,F223__6,F223__7,F223__8,F223__9,F223__10,F223__11,F224__0,F224__1,F224__2,F224__3,F224__4,F224__5,F224__6,F224__7,F224__8,F224__9,F224__10,F224__11,F224__12,F224__13,F224__14,F224__15,F224__16,F224__17
row_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,-0.354591,-0.464038,2.304115,0.734486,1.696395,0.136285,-0.518344,0.502640,-1.852504,-0.500665,-1.416075,1.201521,0.551902,-0.759827,NaN,-0.240522,0.061529,0.561090,-0.171943,-0.431996,0.473508,0.596924,0.819306,1.479061,1.264616,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.380940,-0.499626,-0.418548,1.911725,-0.826130,-1.715371,-0.577091,-1.041486,0.596067,-0.363425,-0.853630,0.674525,0.843058,-0.041438,0.259496,-0.446068,1.433358,0.040099,-1.994062,-0.602324,-0.611391,-1.151884,0.065485,0.352023,-0.843751,...,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0.256023,-1.059874,NaN,0.345678,1.513814,1.243864,-0.509648,-0.800481,-0.115945,0.595777,-0.073235,-1.381605,-0.108676,0.703693,-0.464042,-0.770592,0.483139,-0.636484,-1.305018,-2.089889,0.276761,-1.208640,-0.855769,0.232363,0.215841,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3,-0.728420,-2.432399,-2.453602,-0.020509,0.333397,0.086049,-1.787601,0.667011,0.761564,-2.217847,-0.618973,0.742112,0.494157,0.744673,-0.769181,-0.691020,-1.003026,0.868989,0.464920,NaN,0.499838,0.203723,-0.451576,-1.233499,0.903493,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0.590212,-0.066127,0.468009,-1.096038,0.119399,-1.809710,0.466358,-0.053196,-0.580320,-1.143500,1.338692,1.193330,1.038010,-0.763692,0.513466,1.196402,1.076537,0.486482,0.424305,0.475876,0.426978,-1.668823,-0.843056,1.008980,-0.704152,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,-0.823740,0.285673,0.343307,-0.436747,1.700549,-1.069432,0.819698,-0.168457,-0.429074,0.844075,0.569298,0.235662,0.540328,-0.262106,-1.566722,0.030114,-0.698985,1.176185,-0.621560,-1.042985,-0.340686,0.397049,0.346835,0.142491,-0.161888,...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
999996,-0.769106,-0.387363,-1.227469,0.601183,0.351161,0.219475,-0.530277,0.853452,0.608646,1.648023,-0.040150,-1.235223,0.834497,1.385112,-1.575306,-0.558337,-0.578057,-0.161824,0.507185,-1.523946,0.354188,1.644721,0.809869,-0.483379,0.568550,...,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
999997,0.147534,-0.715276,-0.465049,-1.988941,-1.594535,-1.044882,3.159455,-0.634108,0.890382,-1.212444,-0.874219,0.926165,-0.970726,-1.832919,0.860467,0.141970,-1.507377,0.037872,0.710494,-0.212684,1.961995,0.141009,-0.072367,-0.922849,2.934152,...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
dpkl.info()

In [ ]:
# float 64
dpkl.head()

# ML

In [2]:
%reload_ext cython
import os

os.environ["CC"] = "icx"
os.environ["CXX"] = "icpx"

In [2]:
# define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-W#warnings]
import pickle
import warnings

import numpy as np
# cimport pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearnex import patch_sklearn
from sklearnex._device_offload import _get_device_info_from_daal4py
from sklearnex.ensemble import RandomForestRegressor as rfe
from sklearnex.model_selection import train_test_split

# cpdef load():
#     with open("data.pkl", "rb") as fp:
#         dpkl = pickle.load(fp)
#     return dpkl
# def load():
#     return loading()

In [3]:
%%cython -+
import pickle

import pandas as pd


cpdef sv(df: pd.DataFrame, name: str):
    with open(f'{name}.pkl', 'wb+') as fp:
        pickle.dump(df, fp)
cpdef load():
    with open("data.pkl", "rb") as fp:
        dpkl = pickle.load(fp)
    return dpkl
cpdef pkload(fn):
    with open(fn+'.pkl', "rb") as fp:
        dpkl = pickle.load(fp)
    return dpkl

### TRial 2

In [14]:
# from __main__ import *
%time dpkl = load()

CPU times: user 58.5 ms, sys: 40.3 ms, total: 98.8 ms
Wall time: 102 ms


In [ ]:
from num

In [16]:
# cols_missing = dpkl.missing_cols.copy()
# sv(cols_missing, 'missing_data')
%time cols_missing = pkload('missing_data')

CPU times: user 1.09 ms, sys: 4.7 ms, total: 5.8 ms
Wall time: 5.11 ms


In [17]:
pristine = cols_missing[cols_missing == 0].index

# mf<n> :
# mf : missing_features
# n : count
mf1 = cols_missing[cols_missing == 1]

In [13]:
mf1_data = dpkl.loc[mf1, :].drop(["missing_cols"], axis=1)

In [14]:
mf1_data.shape

(370798, 423)

In [9]:
q = dpkl.to_numpy()
w = mf1.to_numpy()

In [10]:
e = q[w, :423]

In [11]:
e.shape

(370798, 423)

In [1]:
import dpctl
import numpy as np
from numba import njit


@njit
def f1(a, b):
    c = a + b
    return c


global_size = 64
local_size = 32
N = global_size * local_size
a = np.ones(N, dtype=np.float32)
b = np.ones(N, dtype=np.float32)
with dpctl.device_context("level_zero:gpu:0"):
    c = f1(a, b)

TBB Warning: The number of workers is currently limited to 3. The request for 31 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.



In [1]:
import pickle

import dpctl
import numpy as np
import pandas as pd
from numba import njit


def conn():
    with open("data.pkl", "rb") as fp:
        dpkl = pickle.load(fp)
    return dpkl


class DB:
    def __init__(self):
        self.df: pd.DataFrame = conn()
        self.ndf: np.array = self.df.to_numpy()
        # removing the missing_cols

    @staticmethod
    def sv(data: pd.DataFrame, name: str):
        with open(f"{name}.pkl", "wb+") as fp:
            pickle.dump(data, fp)

    @staticmethod
    def pkload(fn):
        with open(fn + ".pkl", "rb") as fp:
            data = pickle.load(fp)
        return data


if __name__ == "__main__":
    pass

In [2]:
D = DB()

In [3]:
e = D.pkload("missing_data")

In [4]:
tm = D.df.drop(["missing_cols"], axis=1)

In [6]:
D.sv(tm, "data")

In [10]:
D.df.shape

(1000000, 423)

In [1]:
from db_store import DB

e = DB()
s = e.data_missing_cols(0)
# e.ndf.shape

TBB Warning: The number of workers is currently limited to 3. The request for 31 workers is ignored. Further requests for more workers will be silently ignored until the limit changes.



In [2]:
import pickle

import dpctl
import numpy as np
import pandas as pd
from numba import njit
from sklearn.preprocessing import StandardScaler

# ww = e.missing_data

In [2]:
s.shape

(185543, 423)

In [2]:
import dpctl

"""Programmatically get a list of the available devices.


The list can be filtered based on backend or device_type.

"""

print("Get a list of all devices:\n")


for d in dpctl.get_devices():

    d.print_device_info()

print("=======================================\n")


print("Get the list of only OpenCL devices:\n")


for d in dpctl.get_devices(backend="opencl"):

    d.print_device_info()


print("=======================================\n")


print("Get all OpenCL CPU devices:\n")


for d in dpctl.get_devices(backend="opencl", device_type="cpu"):

    d.print_device_info()


print("=======================================\n")

Get a list of all devices:

    Name            Intel(R) FPGA Emulation Device
    Driver version  2022.13.3.0.16_160000
    Vendor          Intel(R) Corporation
    Profile         EMBEDDED_PROFILE
    Filter string   opencl:accelerator:0

    Name            11th Gen Intel(R) Core(TM) i3-1115G4 @ 3.00GHz
    Driver version  2022.13.3.0.16_160000
    Vendor          Intel(R) Corporation
    Profile         FULL_PROFILE
    Filter string   opencl:cpu:0

    Name            Intel(R) FPGA Emulation Device
    Driver version  2022.13.3.0.16_160000
    Vendor          Intel(R) Corporation
    Profile         EMBEDDED_PROFILE
    Filter string   opencl:accelerator:1

    Name            11th Gen Intel(R) Core(TM) i3-1115G4 @ 3.00GHz
    Driver version  2022.13.3.0.16_160000
    Vendor          Intel(R) Corporation
    Profile         FULL_PROFILE
    Filter string   opencl:cpu:1

    Name            Intel(R) UHD Graphics [0x9a78]
    Driver version  22.22.23355
    Vendor          Intel(R) 

In [4]:
import sklearnex

In [7]:
import dpctl

In [17]:
dpctl.get_num_devices()

6

In [19]:
%%debug
dpctl._sycl_device_factory

NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(2)<module>()



ipdb>  'c'


'c'


ipdb>  c


In [4]:
e.shape

(1000000,)

In [12]:
type(e)

pandas.core.series.Series

In [13]:
ee = e.to_numpy()

In [17]:
er = np.where(ee > 1)

In [27]:
ee[:20]

array([2, 1, 2, 1, 2, 3, 0, 2, 0, 0, 2, 3, 2, 1, 2, 0, 0, 1, 1, 2],
      dtype=uint8)

In [26]:
ee[ee == 0].shape

(364774,)

In [28]:
xx = np.where(ee == 1)

(array([     1,      3,     13, ..., 999980, 999986, 999992]),)